# Title 1 Enrollment Across the Austin Area

## Hypothesis	
If Austin ISD school enrollment decreases, then the proportion of title 1 students to non title 1 students will increase

## Null	
If Austin ISD school enrollment decreases, then the proportion of title 1 students to non title 1 students will decrease or not change		

![title](AISD.jpg)

# Our Data Sources:

1. CSV files from the TEA(Texas Education Agency) website.
2. GreatSchools.org API
3. census API
4. google maps API


In [4]:
# Dependencies
from keys import googleKey

import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from IPython.display import display
import ipywidgets as widgets
import SchoolExplorer as se

# The Austin Population Heat Map

In [7]:
AustinZipCo_df= pd.read_csv("csv_files/AustinPopulation.csv")

In [9]:

# Store 'Lat' and 'Lng' into  locations 
locations = AustinZipCo_df[["LAT", "LNG"]].astype(float)

# Convert Population to float and store
# HINT: be sure to handle NaN values
population = AustinZipCo_df["Percent Population"].astype(float)
fig4 = gmaps.figure(layout={'width': '1000px','height': '1000px','border': '1px solid black','padding': '1px'})
population_layer = gmaps.heatmap_layer(locations, weights=population, 
                                 dissipating=False, max_intensity=6,
                                 point_radius = 0.05)

fig4.add_layer(population_layer)

fig4

Figure(layout=FigureLayout(border='1px solid black', height='1000px', padding='1px', width='1000px'))

In [5]:
df_full = pd.read_csv('csv_files/AustinSchools.csv')
gmaps.configure(api_key=googleKey.gkey)
df = df_full[['schoolName', 'latitude', 'longitude','enrollment','type']]
df = df.rename(columns={"type":"schoolType"})
df["Percent enrollment"] = 100 * df["enrollment"] / df["enrollment"].sum()
df.fillna(0, inplace=True)
se.SchoolExplorer(df).render()